In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

In [3]:
def paytable(n):
    import numpy as np

    #Generate a 10x...x10 array of 0s
    L = []
    for i in range(n):
        L.append(10)
    L = tuple(L)
    gametable = np.zeros(L, tuple)

    #Iterate through every point
    for strategy in np.ndindex(L):
        payoffs = []
        for i in range(len(strategy)):
            payoffs.append(0)
        #Find the payoff for each player at each point
        for place in range(10):
            distances = []
            for player in range(len(strategy)):
                #Calculate how far each player is from each location
                distances.append(abs(place - strategy[player]))
            mindist = min(distances)
            #Generate a list of players who are closest
            closest = []
            for i, j in enumerate(distances):
                if j == mindist:
                    closest.append(i)
            #Divide up the payoff equally between these players
            sharing = len(closest)
            for i in closest:
                payoffs[i] += 1/sharing
        payoffs = tuple(payoffs)    
        gametable[strategy] = payoffs
    return gametable

In [4]:
threep = paytable(3)

In [5]:
def equilibria(table):
    #Create a list of best responses
    players = table.ndim
    best_response = [[] for i in range(players)]
    dummy = []
    for i in range(players-1):
        dummy.append(10)
    dummy = tuple(dummy)

    #Find all best responses for each player
    for player in range(players):
        for index in np.ndindex(dummy):
            #Find the payoffs for all strategies of a player when the other players strategies are constant
            payoffs = []
            for strat in range(10):
                otherstrats = list(index)
                otherstrats.insert(player, strat)
                otherstrats = tuple(otherstrats)
                payoffs.append(table[otherstrats][player])
            maxi = max(payoffs)
            #Check if a stratagy has the highest pay
            for strat, pay in enumerate(payoffs):
                #If a strategy is a best response append it to a list
                if pay == maxi:
                    otherplays = list(index)
                    otherplays.insert(player, strat)
                    best_response[player].append(otherplays)

    #Find the spaces that are best responses for all players
    common_elements = [value for value in best_response[0] if value in best_response[1]]
    for i in range(players-2):
        common_elements = [value for value in common_elements if value in best_response[i+2]]
    common_set = set()
    #Remove elements that are identical under permutation
    for i in common_elements:
        i.sort()
        i = tuple(i)
        common_set.add(i)
    return common_set

In [6]:
def three_player_function(mixed_strats, paytable):
    answer = mixed_strats
    for player in range(3):
        mixed_pay = 0
        pure_pay = []

        #Calculate the payoff of the mixed strategy
        for strat0, prob0 in enumerate(mixed_strats[0]):
            for strat1, prob1 in enumerate(mixed_strats[1]):
                for strat2, prob2 in enumerate(mixed_strats[2]): 
                    mixed_pay += paytable[strat0, strat1, strat2][player] * prob0 * prob1 * prob2

        #Calculate the payoffs of the pure strategies
        if player == 0:
            for pure_strat in range(10):
                payoff = 0
                for strat1, prob1 in enumerate(mixed_strats[1]):
                    for strat2, prob2 in enumerate(mixed_strats[2]):
                        payoff += paytable[pure_strat, strat1, strat2][player] * prob1 * prob2
                pure_pay.append(payoff)
        elif player == 1:
            for pure_strat in range(10):
                payoff = 0
                for strat0, prob0 in enumerate(mixed_strats[0]):
                    for strat2, prob2 in enumerate(mixed_strats[2]):
                        payoff += paytable[strat0, pure_strat, strat2][player] * prob0 * prob2
                pure_pay.append(payoff)
        elif player == 2:
            for pure_strat in range(10):
                payoff = 0
                for strat1, prob1 in enumerate(mixed_strats[1]):
                    for strat0, prob0 in enumerate(mixed_strats[0]):
                        payoff += paytable[strat0, strat1, pure_strat][player] * prob1 * prob0
                pure_pay.append(payoff)
        
        #See if pure startegies perform better than the mixed
        for strat, pay in enumerate(pure_pay):
            if pay <= mixed_pay:
                pure_pay[strat] = 0
            else:
                pure_pay[strat] = pay - mixed_pay
        summation = sum(pure_pay)

        #Adjust the mixed strategies accordingly
        for pure_strat, weight in enumerate(mixed_strats[player]):
            answer[player][pure_strat] = (weight + pure_pay[pure_strat]) / (1 + summation)

    #Return the adjusted mixed strategies 
    return answer


In [7]:
#Choose a strating point for the iteration
new_strat = [[0.1, 0.2, 0.1, 0, 0, 0.2, 0.1, 0.1, 0.1, 0.1 ], 
             [0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0, 0, 0.1, 0.1 ], 
             [0, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0, 0.1, 0.2 ]]

In [ ]:
#Iterate the function repeatedly
for i in range(90000):
    new_strat = three_player_function(new_strat, threep)
plt.plot(range(1,11),new_strat[0])
plt.plot(range(1,11),new_strat[1])
plt.plot(range(1,11),new_strat[2])
plt.ylim([0, 0.6])
plt.xlim([1,10])
plt.xlabel("Position")
plt.ylabel("Probability")
#plt.savefig("graph_of_strats_rough")

In [ ]:
#Plot the next iteration
new_strat = three_player_function(new_strat, threep)
plt.plot(range(1,11),new_strat[0])
plt.plot(range(1,11),new_strat[1])
plt.plot(range(1,11),new_strat[2])
plt.ylim([0, 0.6])
plt.xlim([1,10])
plt.xlabel("Position")
plt.ylabel("Probability")

In [ ]:
import matplotlib.animation as animation
fig, ax = plt.subplots()

new = three_player_function(new_strat, threep)
line, = ax.plot(range(10), new[1])


def animate(i):
    line.set_ydata(three_player_function(new_strat, threep)[1])  # update the data.
    return line,


ani = animation.FuncAnimation(
    fig, animate, interval=20, blit=True, save_count=2000)
plt.show()
ani.save("movie.gif")

In [ ]:
# x_new, bspline, y_new
p0new = np.linspace(0, 9, 50)
bspline = interpolate.make_interp_spline(range(1,11), new_strat[0])
p0_y = bspline(p0new)

p1new = np.linspace(0, 9, 50)
bspline = interpolate.make_interp_spline(range(1,11), new_strat[1])
p1_y = bspline(p1new)

p2new = np.linspace(0, 9, 50)
bspline = interpolate.make_interp_spline(range(1,11), new_strat[2])
p2_y = bspline(p2new)
# Plot the new data points
plt.plot(p0new, p0_y)
plt.plot(p1new, p1_y)
plt.plot(p2new, p2_y)
plt.ylim([0, 0.6])
plt.xlim([1,10])
plt.xlabel("Position")
plt.ylabel("Probability")
plt.savefig("graph_of_strats")